In [2]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xml.etree import ElementTree as ET
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

# I. Data Profiling
- The data was inspected to see assess the integrity of the data collected.

In [3]:
# Read and inspect data
df = pd.read_csv('stackexchange_data.csv')

print(df.shape)
display(df.info())
df.head()

(2941771, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2941771 entries, 0 to 2941770
Data columns (total 5 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   Unnamed: 0  int64 
 1   XML_Line    int64 
 2   Date        object
 3   Title       object
 4   Tags        object
dtypes: int64(2), object(3)
memory usage: 112.2+ MB


None

,Unnamed: 0,XML_Line,Date,Title,Tags
0,0,24,2008-08-01T12:35:56.917,Check for changes to an SQL Server table?,sql|sql-server|datatable|rdbms
1,0,54,2008-08-01T14:19:52.510,Flat file databases,php|sql|database|flat-file
2,0,68,2008-08-01T15:23:05.190,Decoding T-SQL CAST in C#/VB.NET,c#|sql|vb.net|ascii|hex
3,0,75,2008-08-01T15:50:08.537,ASP.NET Site Maps,sql|asp.net|xml|sitemap
4,0,76,2008-08-01T16:08:52.353,Java lib or app to convert CSV to XML file?,java|xml|csv|data-conversion


In [4]:
# Check for missing values
df.isna().sum()

Unnamed: 0    0
XML_Line      0
Date          0
Title         0
Tags          0
dtype: int64

## Data-Processing

In [5]:
# Create copy of dataframe
df_2 = df.copy()

### 1. Obtain only documents with Python tag
- The scope of the analysis will be limited to only posts with the Python tag.

In [6]:
# Convert the obtained tags to a list
df_2["Tags-2"] = df_2["Tags"].str.split('|')
df_2.head()

,Unnamed: 0,XML_Line,Date,Title,Tags,Tags-2
0,0,24,2008-08-01T12:35:56.917,Check for changes to an SQL Server table?,sql|sql-server|datatable|rdbms,"[sql, sql-server, datatable, rdbms]"
1,0,54,2008-08-01T14:19:52.510,Flat file databases,php|sql|database|flat-file,"[php, sql, database, flat-file]"
2,0,68,2008-08-01T15:23:05.190,Decoding T-SQL CAST in C#/VB.NET,c#|sql|vb.net|ascii|hex,"[c#, sql, vb.net, ascii, hex]"
3,0,75,2008-08-01T15:50:08.537,ASP.NET Site Maps,sql|asp.net|xml|sitemap,"[sql, asp.net, xml, sitemap]"
4,0,76,2008-08-01T16:08:52.353,Java lib or app to convert CSV to XML file?,java|xml|csv|data-conversion,"[java, xml, csv, data-conversion]"


In [6]:
# Filter the new dataframe to those only with python as a tag 
# Then create a column with one tag per row
keywords = ['python']
df_sub = df_2[df_2['Tags'].apply(lambda x: any([k in x for k in keywords]))]
df_sub = df_sub.explode("Tags-2")
df_sub.head()

,Unnamed: 0,XML_Line,Date,Title,Tags,Tags-2
10,0,163,2008-08-02T03:35:55.697,XML Processing in Python,python|xml,python
10,0,163,2008-08-02T03:35:55.697,XML Processing in Python,python|xml,xml
11,0,198,2008-08-02T15:11:16.430,How can I find the full path to a font from it...,python|macos|fonts|photoshop,python
11,0,198,2008-08-02T15:11:16.430,How can I find the full path to a font from it...,python|macos|fonts|photoshop,macos
11,0,198,2008-08-02T15:11:16.430,How can I find the full path to a font from it...,python|macos|fonts|photoshop,fonts


In [7]:
# Insect the filtered dataframe
print(df_sub.shape)
df_sub['Tags-2'].nunique()

(4328086, 6)


22570

A total 22570 unique tags in over 4 million posts were obtained. 

# II. Dimensionality Reduction for Each Year
### 1. Extract the month and year from the data

In [8]:
# Split the date and time portion of the Date column
df_sub[['Year_month', 'Time']] = df_sub["Date"].str.split("T", expand=True)

In [9]:
# Obtain the year and month from the date
df_sub['yr'] = pd.to_datetime(df_sub['Year_month']).dt.year.astype(int)
df_sub['month'] = pd.to_datetime(df_sub['Year_month']).dt.to_period('M')
df_sub.head()

,Unnamed: 0,XML_Line,Date,Title,Tags,Tags-2,Year_month,Time,yr,month
10,0,163,2008-08-02T03:35:55.697,XML Processing in Python,python|xml,python,2008-08-02,03:35:55.697,2008,2008-08
10,0,163,2008-08-02T03:35:55.697,XML Processing in Python,python|xml,xml,2008-08-02,03:35:55.697,2008,2008-08
11,0,198,2008-08-02T15:11:16.430,How can I find the full path to a font from it...,python|macos|fonts|photoshop,python,2008-08-02,15:11:16.430,2008,2008-08
11,0,198,2008-08-02T15:11:16.430,How can I find the full path to a font from it...,python|macos|fonts|photoshop,macos,2008-08-02,15:11:16.430,2008,2008-08
11,0,198,2008-08-02T15:11:16.430,How can I find the full path to a font from it...,python|macos|fonts|photoshop,fonts,2008-08-02,15:11:16.430,2008,2008-08


In [10]:
# Filter the data to only contain the years 2010 to 2019
df_new = df_sub[(df_sub["yr"] >= 2010) & (df_sub["yr"] != 2020)]

In [11]:
# Create a new coloumn that changes the delimeter of the tags to a space
df_new["Tags-3"] = df_new["Tags"].str.replace('|', " ")
df_new.head()

<ipython-input-11-5fc7ba97792d>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_new["Tags-3"] = df_new["Tags"].str.replace('|', " ")
<ipython-input-11-5fc7ba97792d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["Tags-3"] = df_new["Tags"].str.replace('|', " ")


,Unnamed: 0,XML_Line,Date,Title,Tags,Tags-2,Year_month,Time,yr,month,Tags-3
52007,0,1552849,2010-01-01T01:00:23.390,"What does this code mean: ""print >> sys.stderr""",python|syntax,python,2010-01-01,01:00:23.390,2010,2010-01,python syntax
52007,0,1552849,2010-01-01T01:00:23.390,"What does this code mean: ""print >> sys.stderr""",python|syntax,syntax,2010-01-01,01:00:23.390,2010,2010-01,python syntax
52009,0,1552875,2010-01-01T01:24:43.583,Sphinx 0.6.3: The languages module cannot be f...,python|python-sphinx,python,2010-01-01,01:24:43.583,2010,2010-01,python python-sphinx
52009,0,1552875,2010-01-01T01:24:43.583,Sphinx 0.6.3: The languages module cannot be f...,python|python-sphinx,python-sphinx,2010-01-01,01:24:43.583,2010,2010-01,python python-sphinx
52012,0,1552902,2010-01-01T01:51:46.140,"How to print the full NumPy array, without tru...",python|arrays|numpy|options|output-formatting,python,2010-01-01,01:51:46.140,2010,2010-01,python arrays numpy options output-formatting


In [12]:
# Save to csv
#df_new.to_csv("processed_data.csv")

### 2. Split the data into 10 different years

In [13]:
# Read the data - Start here when reopining the notebook
# df_new = pd.read_csv("processed_data.csv")

In [14]:
df_2010 = df_new[df_new["yr"] == 2010]
df_2011 = df_new[df_new["yr"] == 2011]
df_2012 = df_new[df_new["yr"] == 2012]
df_2013 = df_new[df_new["yr"] == 2013]
df_2014 = df_new[df_new["yr"] == 2014]
df_2015 = df_new[df_new["yr"] == 2015]
df_2016 = df_new[df_new["yr"] == 2016]
df_2017 = df_new[df_new["yr"] == 2017]
df_2018 = df_new[df_new["yr"] == 2018]
df_2019 = df_new[df_new["yr"] == 2019]
df_2020 = df_new[df_new["yr"] == 2020]

# Create a list of all dataframes
dfs = [df_2010, df_2011, df_2012, df_2013, df_2014,
       df_2015, df_2016, df_2017, df_2018, df_2019]

### 3. Perform TFIDF

- TFIDF was used to vectorize the tags obtained. 
- A TFIDF matrix was created for each year. 

In [15]:
# Perform TFIDF for every year
vecs = ['vec_2010', 'vec_2011', 'vec_2012', 'vec_2013', 'vec_2014',
        'vec_2015', 'vec_2016', 'vec_2017', 'vec_2018', 'vec_2019']
df_tfidf = []
for i, data in enumerate(dfs):
    vecs[i] = TfidfVectorizer(min_df=500, token_pattern=r"(?u)\S\S+")
    df_tfidf.append(vecs[i].fit_transform(data["Tags-3"]))
    print(df_tfidf[i].shape)

(81481, 66)
(127889, 114)
(193660, 168)
(296503, 250)
(363444, 291)
(432487, 338)
(507073, 367)
(623219, 426)
(675790, 448)
(815848, 507)


In [16]:
# Inspect the top terms obtained for 1 year from TFIDF
terms = vecs[0].get_feature_names()

# sum tfidf frequency of each term through documents
sums = df_tfidf[0].sum(axis=0)

# connecting term to its sums frequency
data = []
for col, term in enumerate(terms):
    data.append((term, sums[0,col]))

ranking = pd.DataFrame(data, columns=['term','rank'])
rank = ranking.sort_values('rank', ascending=False)
print(rank[:50])

                 term          rank
44             python  37810.009108
10             django   8312.146220
17  google-app-engine   3368.361745
27               list   1769.661046
48              regex   1561.945521
55             string   1491.339269
63            windows   1401.152529
35              numpy   1338.699967
26              linux   1265.086576
9          dictionary   1212.075421
13      django-models   1196.878012
4                 c++   1175.721431
23               java   1120.212815
33     multithreading   1092.028562
64           wxpython   1063.562316
34              mysql   1031.502234
57            tkinter    995.024267
30              macos    972.430093
45         python-3.x    969.699040
53         sqlalchemy    938.296224
39                php    893.946288
65                xml    872.055483
19               html    850.815373
41             pylons    818.107676
31         matplotlib    807.398001
24         javascript    803.193631
58            twisted    787

### 4. Perform SVD
- SVD was tested using one year data to assess performance.
- Upon checking, it was observed that limitations on computational capacity allows us to run only a limited amount of model components which reaches approximately 50% of explained variance.  

In [ ]:
# Test run the SVD
svd_model = TruncatedSVD(n_components=100) # captures 50% of variance explained
svd_model.fit(df_tfidf[9])
model = svd_model.fit_transform(df_tfidf[9])
terms = vecs[9].get_feature_names()

pd.DataFrame(model)

In [ ]:
# Check the explained variance ratio obtained for one year
ve = svd_model.explained_variance_ratio_
ve

In [ ]:
# Check the cumulative explained variance ratio obtained for one year
ve.cumsum()

In [ ]:
# Plot the explained variance obtained
fig, ax = plt.subplots()
ax.plot(range(1, len(ve)+1), ve, 'o-', label='individual')
ax.plot(range(1, len(ve)+1), ve.cumsum(), 'o-', label='cumulative')
ax.legend()
ax.set_ylim(0, 1)
ax.set_xlabel('SV')
ax.set_ylabel('variance explained');

In [ ]:
# Check the best features for one SV
best_features = [(terms[i], svd_model.components_[0][i]) 
                 for i in svd_model.components_[0].argsort()[::-1]]
best_features[:10]

### 5. Perform SVD
- SVD was iterated over all years to obtain the SVs per year

In [ ]:
# Obtain the SVD results for all years

df_all = pd.DataFrame()
for i, year in tqdm(enumerate(np.arange(2010, 2020))):
    if i == 0:
        svd_model = TruncatedSVD(n_components=50)
        svd_model.fit(df_tfidf[i])
        terms = vecs[i].get_feature_names()
    else:
        svd_model = TruncatedSVD(n_components=100)
        svd_model.fit(df_tfidf[i])
        terms = vecs[i].get_feature_names()
        
    df_tags = pd.DataFrame()
    d_all = {}
    for i, comp in enumerate(svd_model.components_):   
        if i != 11:
            best_features = [(terms[n], np.abs(svd_model.components_[i][n]))
                 for n in np.abs(svd_model.components_[i].argsort())[::-1]]  
            d_ = {}
            d_["SV_"+str(i)] = best_features  
            d_all.update(d_)
        else:
            break
    df_tags = df_tags.from_dict(d_all)
    df_tags['year'] = year
    df_all = df_all.append(df_tags)
    display(df_all)

In [ ]:
# Save the results to csv
# df_all.to_csv("svd_results_wordlist_v4.csv")

# Appendix: Test

- Results of the sklearn SVD was compared to a user-defined function of truncated SVD to check comparativeness of results

a. Test using user-defined function

In [ ]:
# Create user-defined function
def truncated_svd(X):
    """Given input design matrix, return q, sigma, p and normalized sum of
    squared distance from the origin.
    """
    q, sv, p = np.linalg.svd(X)
    s = np.diag(sv)
    p = p.T
    ssd = (sv**2)/sum(sv**2)
    return q, s, p, ssd

In [ ]:
# Run TFIDF for a smaller dataframe
vectorizer = TfidfVectorizer()
test = vectorizer.fit_transform(df_2010["Tags-3"].head(100))
test

In [ ]:
# Run the user-defined truncated svd
q_ng, s_ng, p_ng, nssd_ng = truncated_svd(test.toarray())
len(nssd_ng)

In [ ]:
# Visualize explained ratio results
fig, ax = plt.subplots()
ax.plot(range(1, len(nssd_ng)+1), nssd_ng, '-', label='individual')
ax.set_xlim(0, len(nssd_ng)+1)
ax.set_xlabel('SV')
ax.set_ylabel('variance explained')
ax = ax.twinx()
ax.plot(range(1, len(nssd_ng)+1), nssd_ng.cumsum(), 'r-', label='cumulative')
ax.axhline(0.9, ls='--', color='g')
ax.axvline(662, ls='--', color='g')
ax.set_ylabel('cumulative variance explained');

In [ ]:
# Visualize best features results
feature_names = vectorizer.get_feature_names()
for i in range(10):
    fig, ax = plt.subplots()
    order = np.argsort(np.abs(p_ng[:, i]))[-10:]
    ax.barh([feature_names[o] for o in order], p_ng[order, i])
    ax.set_title(f'SV{i+1}')

b. Test and compare results using sklearn

In [ ]:
# Perform SVD using sklearn
svd_model = TruncatedSVD(algorithm='arpack', n_components=65) # captures 50% of variance explained
svd_model.fit(test)
terms = vectorizer.get_feature_names()

In [ ]:
# Check and compare best features with the user-defined function
best_features = [(terms[i], np.abs(svd_model.components_[1][i]))
                 for i in np.abs(svd_model.components_[1].argsort())[::-1]]
best_features[:10]

In [ ]:
# Check and compare explained variance with the user-defined function
ve = svd_model.explained_variance_ratio_
fig, ax = plt.subplots()
ax.plot(range(1, len(ve)+1), ve, 'o-', label='individual')
ax.plot(range(1, len(ve)+1), ve.cumsum(), 'o-', label='cumulative')
ax.legend()
ax.set_ylim(0, 1)
ax.set_xlabel('SV')
ax.set_ylabel('variance explained');